In [1]:
import argparse
import numpy as np
from PIL import Image
import pickle
from utils import *
import os
import matplotlib as mpl
mpl.use('Agg')

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

import numpy as np
import argparse
import math
import os
import torch
import torch.optim as optim
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.optim as optim

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

def load_databatch(data_folder, idx, img_size=32):
    data_file = os.path.join(data_folder, 'train_data_batch_')

    d = unpickle(data_file + str(idx))
    x = d['data']
    y = d['labels']
    mean_image = d['mean']

    x = x/np.float32(255)
    mean_image = mean_image/np.float32(255)

    # Labels are indexed from 1, shift it so that indexes start at 0
    y = [i-1 for i in y]
    data_size = x.shape[0]

    x -= mean_image

    img_size2 = img_size * img_size

    x = np.dstack((x[:, :img_size2], x[:, img_size2:2*img_size2], x[:, 2*img_size2:]))
    x = x.reshape((x.shape[0], img_size, img_size, 3)).transpose(0, 3, 1, 2)

    # create mirrored images
    X_train = x[0:data_size, :, :, :]
    Y_train = y[0:data_size]
    X_train_flip = X_train[:, :, :, ::-1]
    Y_train_flip = Y_train
    X_train = np.concatenate((X_train, X_train_flip), axis=0)
    Y_train = np.concatenate((Y_train, Y_train_flip), axis=0)

    return dict(
        X_train=(X_train), # lasagne.utils.floatX(X_train),
        Y_train=Y_train.astype('int32'),
        mean=mean_image)


def load_data(input_file, train = True):

    d = unpickle(input_file)
    x = d['data']
    y = d['labels']
    if train:
        mean_image = d['mean']

    x = np.divide(x,np.float32(255))
    mean_image = mean_image/np.float32(255)

    # Labels are indexed from 1, shift it so that indexes start at 0
    y = [i-1 for i in y]

    x -= mean_image

    x = np.dstack((x[:, :1024], x[:, 1024:2048], x[:, 2048:]))
    x = x.reshape((x.shape[0], 32, 32, 3))

    return x, y

def get_train_dat(p):

    path = p
    counter = 0

    for i in os.listdir(path):
        counter += 1
        print(counter)
        total_path = path + i

        x,y = load_data(total_path)

        if counter == 1:
            x_train, y_train = x, y
        else:
            x_train = np.append(x_train, x, axis = 0)
            y_train = y_train + y
    return x_train, y_train

def create_tuples(x,y):
    samps = []

    for i in np.arange(0,len(x),1):
        t = torch.Tensor(x[i]).view(3,32,32)
        tup = (t, int(y[i]))
        samps.append(tup)
    return samps

def get_test_dat(p):
    x_test , y_test = load_data(p, False)
    dat_test = create_tuples(x_test, y_test)
    return dat_test

def create_imnet_train_test(path_train, path_test):
    x_train, y_train = get_train_dat(path_train)
    dat_train = create_tuples(x_train, y_train)
    trainset = torch.utils.data.DataLoader(dat_train, batch_size = 10, shuffle = True)

    x_test,y_test = get_test_dat(path_test)
    dat_test = create_tuples(x_test, y_test)
    testset = torch.utils.data.DataLoader(dat_test, batch_size = 10, shuffle = True)

    return trainset, testset



In [3]:
path_train = '/Users/dj/Desktop/MT_Desktop/ImageNet/Imagenet32_train/'
path_test = '/Users/dj/Desktop/MT_Desktop/ImageNet/val_data'

trainset, testset = create_imnet_train_test(path_train= path_train, path_test=path_test)

1
2
3
4
5
6
7
8
9
10


UnboundLocalError: local variable 'mean_image' referenced before assignment

In [4]:
type(trainset)

NameError: name 'trainset' is not defined

In [6]:
# torch.save(trainset, '/Users/dj/Desktop/imnet_train.pth')

In [7]:
# torch.save(testset, '/Users/dj/Desktop/imnet_test.pth')

In [11]:
# torch.load('/Users/dj/Desktop/imnet_test.pth')

In [ ]:
current_path = os.getcwd()
dat_path = (current_path + '/data/')

if True:
    dir = 'torch_imnet'
    final_dat_path = (dat_path + dir)
    if not os.path.exists(final_dat_path):
        os.mkdir(final_dat_path)
    torch.save(trainset, os.path.join(final_dat_path, "imnet_train.pth"))
    torch.save(testset, os.path.join(final_dat_path, "imnet_test.pth"))